# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [33]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [18]:
weather_data_df = pd.read_csv("output_data")
weather_data_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Temp,Lat,Lng,Wind Speed
0,0,rzhavki,100,RU,1596160543,81,51.80,56.00,37.25,6.71
1,1,rikitea,100,PF,1596160596,89,73.17,-23.12,-134.97,11.48
2,2,nikolskoye,40,RU,1596160596,93,57.00,59.70,30.79,4.47
3,3,albany,71,US,1596160596,95,75.00,42.60,-73.97,7.16
4,4,avesta,7,SE,1596160596,90,55.99,60.15,16.17,5.01
...,...,...,...,...,...,...,...,...,...,...
560,560,mersing,40,MY,1596160618,94,73.40,2.43,103.84,3.36
561,561,sinnamary,44,GF,1596160618,81,78.44,5.38,-52.95,4.59
562,562,mahebourg,75,MU,1596160618,83,71.60,-20.41,57.70,3.36
563,563,southbridge,100,US,1596160618,76,72.00,42.08,-72.03,1.52


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [19]:
# configure gmaps
gmaps.configure(api_key=g_key)

In [20]:
# Use the Lat and Lng as locations and Humidity as the weight.
locations = weather_data_df[["Lat", "Lng"]]
humidity = weather_data_df["Humidity"]

In [27]:
# add heatmap layer to map
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [30]:
# Narrow down the cities to fit weather conditions
# Temperature lower than 80 degrees but higher than 70, wind speed less than 10 mph, zero cloudiness

narrow_city_df = weather_data_df.loc[(weather_data_df["Wind Speed"] <= 10) & (weather_data_df["Cloudiness"] == 0) & \
                                   (weather_data_df["Temp"] >= 70) & (weather_data_df["Temp"] <= 80)].dropna()
narrow_city_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Temp,Lat,Lng,Wind Speed
19,19,kisangani,0,CD,1596160597,80,71.73,0.52,25.20,2.01
85,85,korla,0,CN,1596160599,69,73.99,41.76,86.15,1.01
111,111,uhlove,0,UA,1596160600,73,72.72,44.82,33.60,1.19
157,157,charters towers,0,AU,1596160602,33,70.43,-20.10,146.27,9.84
159,159,manono,0,CD,1596160602,40,72.09,-7.30,27.42,6.02
170,170,loralai,0,PK,1596160602,58,78.44,30.37,68.60,6.40
177,177,camrose,0,CA,1596160602,58,75.00,53.02,-112.84,5.99
237,237,sabzevar,0,IR,1596160605,55,72.34,36.21,57.68,8.34
259,259,karratha,0,AU,1596160606,24,76.89,-20.74,116.85,8.30
296,296,maturin,0,VE,1596160541,93,75.16,9.75,-63.18,4.72


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [31]:
# create hodel_df + add column to df
hotel_df = narrow_city_df.loc[:,["City","Country", "Lat", "Lng"]]
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
19,kisangani,CD,0.52,25.20,
85,korla,CN,41.76,86.15,
111,uhlove,UA,44.82,33.60,
157,charters towers,AU,-20.10,146.27,
159,manono,CD,-7.30,27.42,
170,loralai,PK,30.37,68.60,
177,camrose,CA,53.02,-112.84,
237,sabzevar,IR,36.21,57.68,
259,karratha,AU,-20.74,116.85,
296,maturin,VE,9.75,-63.18,


In [37]:
# Set parameters to search for hotels with 5000 meters
params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


In [39]:
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city = row["City"]
    
    params["location"] = f"{lat},{lng}"
    # api request
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    # save the closest hotel name to dataframe
    try:
        print(f"Closest hotel in {city} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # show missing field if no hotel avail
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")


Closest hotel in kisangani is Palm Beach Hotel.
Closest hotel in korla is Loulan Hotel.
Closest hotel in uhlove is Our Sail.
Closest hotel in charters towers is Charters Towers Motel.
Closest hotel in manono is Hotel hewa bora.
Closest hotel in loralai is Loralai Hotel.
Closest hotel in camrose is Camrose Le Chateau Hotel.
Closest hotel in sabzevar is Sabzevar Tourism hotel.
Closest hotel in karratha is Karratha International Hotel.
Closest hotel in maturin is Residencias Guaritos I.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Closest hotel in changji is Garden Hotel.
Closest hotel in caravelas is Mar Doce de Abrolhos.
Closest hotel in jalu is Jalu Hotel.
Closest hotel in santa isabel do rio negro is Hotel Ayuana.
Missing field/result... skipping.
Closest hotel in goderich is Samuels Hotel.
Closest hotel in kindu is Valencia City Hotel.
Closest hotel in parys is Hôtel La Bourdonnais Paris.


In [40]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
19,kisangani,CD,0.52,25.20,Palm Beach Hotel
85,korla,CN,41.76,86.15,Loulan Hotel
111,uhlove,UA,44.82,33.60,Our Sail
157,charters towers,AU,-20.10,146.27,Charters Towers Motel
159,manono,CD,-7.30,27.42,Hotel hewa bora
170,loralai,PK,30.37,68.60,Loralai Hotel
177,camrose,CA,53.02,-112.84,Camrose Le Chateau Hotel
237,sabzevar,IR,36.21,57.68,Sabzevar Tourism hotel
259,karratha,AU,-20.74,116.85,Karratha International Hotel
296,maturin,VE,9.75,-63.18,Residencias Guaritos I


In [43]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [44]:
# Add marker layer ontop of heat map
marker = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker)
fig

Figure(layout=FigureLayout(height='420px'))